In [1]:
import geopandas as gpd
import pandas as pd

# Load processed GeoJSON
gdf = gpd.read_file("../data/processed/india_dengue_state_year.geojson")

# Enforce numeric types (safe practice)
gdf["cases"] = pd.to_numeric(gdf["cases"], errors="coerce")

# We only need unique state geometries
states_gdf = gdf[["state", "geometry"]].drop_duplicates().reset_index(drop=True)

print("Number of states:", len(states_gdf))
states_gdf.head()


Number of states: 41


,state,geometry
0,None,"MULTIPOLYGON (((93.7908 6.8514, 93.7872 6.8478..."
1,None,"MULTIPOLYGON (((78.7395 13.0455, 78.7255 13.05..."
2,None,"MULTIPOLYGON (((95.3683 27.1074, 95.3737 27.10..."
3,None,"MULTIPOLYGON (((94.1912 27.4963, 94.1869 27.49..."
4,Assam,"MULTIPOLYGON (((92.7545 24.5011, 92.7436 24.47..."


In [2]:
print(states_gdf.crs)


EPSG:4326


In [3]:
adjacency = []

for i, row_i in states_gdf.iterrows():
    state_i = row_i["state"]
    geom_i = row_i["geometry"]
    
    for j, row_j in states_gdf.iterrows():
        state_j = row_j["state"]
        geom_j = row_j["geometry"]
        
        # Avoid self-comparison
        if state_i == state_j:
            continue
        
        # Queen contiguity: share boundary
        if geom_i.touches(geom_j):
            adjacency.append({
                "state": state_i,
                "neighbor": state_j
            })

adj_df = pd.DataFrame(adjacency)
adj_df.head()


,state,neighbor
0,None,Nagaland
1,None,Assam
2,Assam,None
3,Assam,Manipur
4,Assam,Mizoram


In [4]:
# Number of adjacency pairs
print("Adjacency pairs:", len(adj_df))

# Check a few states
adj_df[adj_df["state"] == "Bihar"]


Adjacency pairs: 30


,state,neighbor


In [5]:
# Ensure symmetry
adj_df_rev = adj_df.rename(columns={"state": "neighbor", "neighbor": "state"})
adj_df = pd.concat([adj_df, adj_df_rev]).drop_duplicates().reset_index(drop=True)

adj_df.head()


,state,neighbor
0,None,Nagaland
1,None,Assam
2,Assam,None
3,Assam,Manipur
4,Assam,Mizoram


In [6]:
adj_df.to_csv("../data/processed/state_adjacency.csv", index=False)
